# Lesson 6 - Creating an A2A Sequential Chain Agent with ADK

Now that you have two active agents (Policy Agent and Research Agent), you will orchestrate them. In this lesson, you will use Google ADK's `SequentialAgent` to create a workflow where a user's query is processed by the Research Agent first, and then the Policy Agent, or vice versa, based on the sequential definition. You will use `RemoteA2aAgent` to connect to the servers you started in previous lessons.

```mermaid
graph LR
    classDef orchestrator fill:#f9f,stroke:#333,stroke-width:2px;
    classDef agent fill:#e1f5fe,stroke:#0277bd,stroke-width:2px;
    classDef tool fill:#fff3e0,stroke:#ef6c00,stroke-width:1px,stroke-dasharray: 5 5;

    subgraph Sequential [Sequential Agent]
        direction LR
        
        HealthcareAgent["<b>Healthcare Research<br/>Agent</b>"]
        PolicyAgent["<b>Policy<br/>Agent</b>"]
        
        HealthcareAgent --> PolicyAgent
    end

    class HealthcareAgent,PolicyAgent agent;
```

In [75]:
import logging
import warnings

logging.disable(level=logging.WARNING)
warnings.filterwarnings("ignore")

## 6.1. Start the A2A Servers

Ensure both the Policy Agent (Terminal 1) and Research Agent (Terminal 2) are running.

In [76]:
import os

from IPython.display import IFrame

url = os.environ.get("DLAI_LOCAL_URL").format(port=8888)
# Terminal 1: uv run a2a_policy_agent.py
IFrame(f"{url}terminals/1", width=800, height=200)

In [77]:
# Terminal 2: uv run a2a_research_agent.py
IFrame(f"{url}terminals/2", width=800, height=200)

## 6.2. Define the Sequential Workflow

Here you will:
1.  Define two `RemoteA2aAgent` instances. These act as proxies that know how to communicate with your running servers via A2A.
2.  Create a `SequentialAgent` named `root_agent`. This agent contains the logic to route the workflow through the sub-agents.
3.  Use an `InMemoryRunner` to execute the flow with a specific prompt.


In [78]:
import os

from IPython.display import Markdown, display
from dotenv import load_dotenv
from google.adk.agents import SequentialAgent
from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
)
from google.adk.runners import InMemoryRunner

In [79]:
load_dotenv()
host = os.environ.get("AGENT_HOST")
policy_port = os.environ.get("POLICY_AGENT_PORT")
research_port = os.environ.get("RESEARCH_AGENT_PORT")

In [80]:
policy_agent = RemoteA2aAgent(
    name="policy_agent",
    agent_card=f"http://{host}:{policy_port}",
)
print("\tℹ️", f"{policy_agent.name} initialized")

	ℹ️ policy_agent initialized


In [81]:
health_research_agent = RemoteA2aAgent(
    name="health_research_agent",
    agent_card=f"http://{host}:{research_port}",
)
print("\tℹ️", f"{health_research_agent.name} initialized")

	ℹ️ health_research_agent initialized


In [82]:
root_agent = SequentialAgent(
    name="root_agent",
    description="Healthcare Routing Agent",
    sub_agents=[
        health_research_agent,
        policy_agent,
    ],
)
print("\tℹ️", f"{root_agent.name} initialized")

	ℹ️ root_agent initialized


## 6.3. Run the Sequential Chain

The `InMemoryRunner` executes the agent. The prompt will trigger the Research Agent to find general info, and then (sequentially) the Policy Agent to check coverage details.


In [83]:
prompt = "How can I get mental health therapy?"

In [84]:
print("Running Healthcare Workflow Agent")

runner = InMemoryRunner(root_agent)

for event in await runner.run_debug(prompt, quiet=True):
    if event.is_final_response() and event.content:
        display(Markdown(event.content.parts[0].text))

Running Healthcare Workflow Agent


Getting mental health therapy involves a few key steps: identifying what kind of help you need, checking payment options (insurance or out-of-pocket), and finding a provider who is a good fit.

### **1. Immediate Help (Crisis Resources)**
If you or someone you know is in immediate danger or needs urgent support, use these free, confidential resources available 24/7:
*   **988 Suicide & Crisis Lifeline:** Call or text **988** (USA).
*   **Crisis Text Line:** Text **HOME** to **741741**.
*   **Emergency Services:** Call **911** or go to the nearest emergency room.

---

### **2. How to Find a Therapist**
There are several ways to find a mental health professional depending on your situation:

*   **Through Insurance:**
    *   **Call the number on the back of your card:** Ask for a list of in-network mental health providers.
    *   **Use their online portal:** Most insurers have a "Find a Doctor" tool where you can filter by "Behavioral Health" or "Mental Health."
    *   **Online Platforms:** Services like **Headway**, **Grow Therapy**, and **Alma** allow you to search for therapists who specifically accept your insurance plan.

*   **Online Directories (Great for filtering by specialty):**
    *   **Psychology Today:** One of the most comprehensive directories. You can filter by zip code, insurance, gender, and specific issues (e.g., anxiety, trauma).
    *   **GoodTherapy:** Similar to Psychology Today, with a focus on educational resources.
    *   **Therapy for Black Girls / Therapy for Latinx:** Specialized directories for finding culturally competent care.
    *   **Open Path Collective:** Connects you with therapists offering sessions at a reduced rate (typically $40-$70) if you don't have insurance.

*   **Referrals:**
    *   Ask your primary care doctor (PCP) for a referral.
    *   Ask friends or family if they recommend someone.

---

### **3. Choosing the Right Professional**
"Therapist" is a general term. Different professionals have different training:

| Professional Type | Degree | Best For... |
| :--- | :--- | :--- |
| **Psychiatrist** | MD or DO | **Medication management.** They are medical doctors who can prescribe medicine. Some do talk therapy, but most focus on medication. |
| **Psychologist** | PhD or PsyD | **Testing & Therapy.** They have a doctorate and often specialize in psychological testing (e.g., for ADHD/Autism) and deep talk therapy. |
| **Licensed Clinical Social Worker (LCSW)** | Master's (MSW) | **Practical support & Therapy.** Trained in talk therapy and often help with connecting you to community resources. |
| **Licensed Professional Counselor (LPC/LMHC)** | Master's (MA/MS) | **Talk Therapy.** Focused on helping you find solutions to life challenges and mental health symptoms. |

---

### **4. Common Types of Therapy**
When searching, you might see these terms. Choose one that sounds like it fits your needs:
*   **CBT (Cognitive Behavioral Therapy):** Good for anxiety and depression; focuses on changing negative thought patterns.
*   **DBT (Dialectical Behavior Therapy):** Helpful for intense emotions, self-harm, or borderline personality disorder; teaches coping skills.
*   **EMDR (Eye Movement Desensitization and Reprocessing):** Specifically designed to treat trauma and PTSD.
*   **Family/Couples Therapy:** Focuses on relationship dynamics and communication.

---

### **5. If You Don’t Have Insurance or Have a Low Budget**
*   **Sliding Scale:** Many private therapists offer a "sliding scale" fee, meaning they adjust the price based on your income. Ask about this before your first appointment.
*   **University Clinics:** If you live near a university with a psychology or social work program, they often have clinics where graduate students provide therapy at a very low cost under supervision.
*   **Community Health Centers:** Federally Qualified Health Centers (FQHCs) often provide mental health services on a "pay what you can" basis.
*   **Employee Assistance Program (EAP):** Check if your employer offers an EAP. These programs often provide a few free counseling sessions to employees and their families.

### **Next Steps**
1.  **Make a list** of 3-5 potential therapists.
2.  **Contact them** (email is often easier) and ask: *"Are you accepting new clients?"* and *"Do you accept my insurance?"*
3.  **Schedule a consultation.** Many offer a free 15-minute phone call to see if you "click." Trust your gut—feeling comfortable with your therapist is the most important factor for success.

Based on the insurance document provided, here's what your plan covers for mental health therapy:

## Mental Health Coverage Under Your Plan

**Outpatient Mental Health Services:**
- **In-Network Provider:** 10% coinsurance after deductible
- **Out-of-Network Provider:** 30% coinsurance after deductible

This includes:
- Office visits
- Other outpatient services

**Inpatient Mental Health Services:**
- **In-Network Provider:** 10% coinsurance after deductible (plus 10% coinsurance for inpatient physician fees)
- **Out-of-Network Provider:** 30% coinsurance after deductible (plus 30% coinsurance for inpatient physician fees)

## Key Details About Your Coverage

1. **Deductible:** You must first meet your deductible (\\$1,700 for individual/\\$3,400 for family with in-network providers) before the plan starts paying its share.

2. **No Referral Required:** You can see a mental health specialist without needing a referral from your primary care doctor.

3. **Network Providers:** To pay the least (10% coinsurance), use an in-network provider. Your plan uses:
   - NY Blue Access PPO (if you're in NY, excluding Suffolk County)
   - GA Blue Open Access POS (if you're in GA)
   - Blue Card PPO (all other areas)

4. **Find Providers:** Visit **includedhealth.com/google** or call **(855) 431-5540** for a list of in-network mental health providers.

The coverage applies to behavioral health and substance abuse services as well.

## 6.4. Resources

- [Google ADK Sequential Agents](https://google.github.io/adk-docs/agents/workflow-agents/sequential-agents/)

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>
</div>
